In [ ]:
import torch
import time
import re

In [ ]:
 from europarl_preprocessing import write_new_files

In [ ]:
import os

tsz = 500000
vate_size = int(tsz/4)
write_new_files(tsz, vate_size, vate_size, max_sentence_length=50)

def get_number_lines(path):
    with open(path, 'r') as file:
        lines = file.readlines()
    return len(lines)        

path = os.path.join('europarl','Europarl.en-es.en')
get_number_lines(path)

In [ ]:
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
directory = 'europarl'

def regexification(sentences):
    regex = [r"[^A-Za-z0-9]+|[a-zA-Z][0-9]", r"(?<!\d)[0]\d*(?!\d)", r"\s+", r"[0-9]+"]
    for r in regex:
        sentences = list(map(lambda sentence: re.sub(r, " ", sentence), sentences))
    return sentences

def get_files_paths(directory):
    train_txt = 'train_sentences.txt'    
    valid_txt = 'valid_sentences.txt'
    test_txt = 'test_sentences.txt'
    train_path = os.path.join(directory,train_txt)    
    valid_path = os.path.join(directory,valid_txt)
    test_path = os.path.join(directory,test_txt)
    return train_path, valid_path, test_path

def get_sentences_from_file(filename):
    sentences = []
    with open(filename, mode='rt', encoding='utf-8') as file_object:
        for line in file_object:
            sentences.append(line)    
    return sentences

def get_clip_embeddings(sentences, batch_size=32):
    tokenized_text = clip.tokenize(sentences).to(device)
    with torch.no_grad():
        clip_embeddings_list = []
        for i in range(0,tokenized_text.size()[0],batch_size):
            tok_batch = tokenized_text[i:i+batch_size]
            clip_embeddings_batch = clip_model.encode_text(tok_batch).to(device)
            for unity in clip_embeddings_batch:
                clip_embeddings_list.append(unity)
    final_emb = torch.stack(clip_embeddings_list)
    return final_emb

In [ ]:
train_file, valid_file, test_file = get_files_paths(directory)
train_len = get_number_lines(train_file)
valid_len = get_number_lines(valid_file)
test_len = get_number_lines(test_file)
print([train_len, valid_len,test_len])

In [ ]:
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("done")

In [ ]:
train_sentences = regexification(get_sentences_from_file(train_file))
valid_sentences = regexification(get_sentences_from_file(valid_file))
train_clip_embeddings = get_clip_embeddings(train_sentences)
valid_clip_embeddings = get_clip_embeddings(valid_sentences)